In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch

%matplotlib inline
import numpy as np
import sys
 
# setting path
sys.path.append('../.')
from giagrad.tensor import Tensor
from giagrad.mlops import Softmax, LogSoftmax
from giagrad.nn.layers import Linear
from giagrad.nn.loss import CrossEntropyLoss
from giagrad.optim import SGD
import torch.optim as toptim

In [2]:
X = torch.randn(2, 10)

## Pytorch

In [3]:
l1t = torch.nn.Linear(10, 10)
l2t = torch.nn.Linear(10, 10)
l3t = torch.nn.Linear(10,  1)

In [6]:
x = l1t(X)
x = l2t(x)
yt = l3t(x).mean()
yt

tensor(0.0106, grad_fn=<MeanBackward0>)

In [7]:
yt.backward()

In [8]:
# GRADS
l3t.weight.grad

tensor([[ 0.2884,  0.3311,  0.0896, -0.1128,  0.1544,  0.2177,  0.3075, -0.2359,
          0.0673, -0.2024]])

In [9]:
l2t.weight.grad

tensor([[-5.6430e-02,  1.4565e-02,  5.7088e-02,  3.3552e-03, -1.6401e-01,
         -2.5608e-02,  7.4786e-02,  4.7204e-02, -9.0295e-02, -6.4606e-02],
        [ 1.2455e-03, -3.2149e-04, -1.2601e-03, -7.4057e-05,  3.6202e-03,
          5.6524e-04, -1.6507e-03, -1.0419e-03,  1.9930e-03,  1.4260e-03],
        [ 1.8835e-02, -4.8615e-03, -1.9054e-02, -1.1199e-03,  5.4743e-02,
          8.5472e-03, -2.4961e-02, -1.5755e-02,  3.0138e-02,  2.1564e-02],
        [-2.8384e-02,  7.3263e-03,  2.8715e-02,  1.6876e-03, -8.2497e-02,
         -1.2881e-02,  3.7617e-02,  2.3743e-02, -4.5417e-02, -3.2496e-02],
        [-4.4793e-03,  1.1562e-03,  4.5315e-03,  2.6633e-04, -1.3019e-02,
         -2.0327e-03,  5.9364e-03,  3.7470e-03, -7.1674e-03, -5.1283e-03],
        [-6.1189e-03,  1.5794e-03,  6.1902e-03,  3.6381e-04, -1.7784e-02,
         -2.7768e-03,  8.1093e-03,  5.1185e-03, -9.7909e-03, -7.0055e-03],
        [ 2.2423e-02, -5.7877e-03, -2.2684e-02, -1.3332e-03,  6.5172e-02,
          1.0176e-02, -2.9717e-0

In [10]:
l1t.weight.grad

tensor([[-0.0854,  0.0159, -0.0457, -0.0504,  0.0710, -0.0576,  0.0061,  0.0410,
          0.0514, -0.1024],
        [-0.0954,  0.0178, -0.0511, -0.0563,  0.0793, -0.0643,  0.0068,  0.0458,
          0.0574, -0.1144],
        [ 0.1390, -0.0259,  0.0744,  0.0821, -0.1157,  0.0938, -0.0099, -0.0668,
         -0.0837,  0.1668],
        [ 0.0871, -0.0162,  0.0466,  0.0514, -0.0725,  0.0588, -0.0062, -0.0419,
         -0.0524,  0.1045],
        [-0.0910,  0.0170, -0.0487, -0.0537,  0.0757, -0.0613,  0.0065,  0.0437,
          0.0547, -0.1091],
        [ 0.0302, -0.0056,  0.0162,  0.0178, -0.0251,  0.0203, -0.0021, -0.0145,
         -0.0182,  0.0362],
        [ 0.0590, -0.0110,  0.0316,  0.0349, -0.0491,  0.0398, -0.0042, -0.0284,
         -0.0355,  0.0708],
        [-0.0304,  0.0057, -0.0163, -0.0179,  0.0253, -0.0205,  0.0022,  0.0146,
          0.0183, -0.0364],
        [-0.1094,  0.0204, -0.0586, -0.0646,  0.0910, -0.0738,  0.0078,  0.0526,
          0.0658, -0.1312],
        [ 0.0156, -

## Giagrad

In [11]:
X = X.detach().numpy()

In [12]:
l1g = Linear(10, 10)
l1g.w.data = l1t.weight.detach().numpy()
l1g.b.data = np.expand_dims(l1t.bias.detach().numpy(), axis=0)

l2g = Linear(10, 10)
l2g.w.data = l2t.weight.detach().numpy()
l2g.b.data = np.expand_dims(l2t.bias.detach().numpy(), axis=0)

l3g = Linear(10, 1)
l3g.w.data = l3t.weight.detach().numpy()
l3g.b.data = np.expand_dims(l3t.bias.detach().numpy(), axis=0)

In [13]:
x = l1g(X)
x = l2g(x)
yg = l3g(x).mean()
yg

0.010628916

In [14]:
yg.backward()

ValueError: non-broadcastable output operand with shape (1,10) doesn't match the broadcast shape (2,10)

In [17]:
# GRADS
l3g.w.grad

array([[ 0.2883718 ,  0.33112568,  0.08960211, -0.11284292,  0.15439382,
         0.21765035,  0.307536  , -0.23586552,  0.06727789, -0.20235609]],
      dtype=float32)

In [ ]:
l2g.w.grad

In [ ]:
l1g.w.grad

In [ ]:
optimizer = SGD(sum([l.parameters() for l in [l1g, l2g, l3g]], []), lr=0.1, momentum=0.9)

## Optimizer

In [ ]:
l3g.w

In [ ]:
l3t.weight

In [ ]:
optimizer.step()

In [ ]:
params = sum([[l.weight, l.bias] for l in [l1t, l2t, l3t]], [])
optimizer = torch.optim.SGD(params, lr=0.1, momentum=0.9)
optimizer.step()

In [ ]:
l3g.w

In [ ]:
l3t.weight